<a href="https://colab.research.google.com/github/imhyunho99/python_vision_with.Prof.Ryu/blob/main/Sivir_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Download and Learning

In [ ]:
#Roboflow version 3 dataset export

!curl -L "https://app.roboflow.com/ds/va7RXfq8gC?key=WSe7eFFav1" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip
#roboflow에서 학습시킨 데이터, 모델 Export and download

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   901  100   901    0     0   2545      0 --:--:-- --:--:-- --:--:--  2545
100  185M  100  185M    0     0  22.3M      0  0:00:08  0:00:08 --:--:-- 25.2M
Archive:  roboflow.zip
 extracting: README.dataset.txt      
 extracting: README.roboflow.txt     
 extracting: data.yaml               
   creating: test/
   creating: test/images/
 extracting: test/images/-22_png.rf.f17b799d721ded7f1177087e23d5aed8.jpg  
 extracting: test/images/-23_png.rf.e899e8e37c05e16cd6d1d3d60ccfaab3.jpg  
 extracting: test/images/-31_png.rf.711b586ff5678be1156bb5d54e5530f9.jpg  
 extracting: test/images/-6_png.rf.4096bfa7dddebf636a247e31145c96dc.jpg  
 extracting: test/images/16_jpg.rf.b1129a94c1f8d21eb26424e2766211ea.jpg  
 extracting: test/images/19_jpg.rf.21f75bd8434b95b3ea92372d07b76005.jpg  
 extracting: test/images/23_jpg.rf.5ebb8e2e895d916ddcaa

In [ ]:
#Yolov5 git commit

%cd /content
!git clone https://github.com/ultralytics/yolov5.git
#github yolo v5 clone

/content
Cloning into 'yolov5'...
remote: Enumerating objects: 15637, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 15637 (delta 123), reused 173 (delta 101), pack-reused 15393
Receiving objects: 100% (15637/15637), 14.65 MiB | 23.80 MiB/s, done.
Resolving deltas: 100% (10647/10647), done.


In [ ]:
#yolo-5 req install

%cd /content/yolov5/
!pip install -r requirements.txt

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
#실행전에 dataset 이라는 폴더를 만들어서 train, test, data.yaml 등 yolov5 제외 모든 파일, 폴더 전부 집어 넣기기

%mkdir dataset

In [ ]:
%mv '/content/test' '/content/dataset'
%mv '/content/train' '/content/dataset'
%mv '/content/valid' '/content/dataset'
%mv '/content/README.dataset.txt' '/content/dataset'
%mv '/content/README.roboflow.txt' '/content/dataset'
%mv '/content/data.yaml' '/content/dataset'

In [ ]:
%cat /content/dataset/data.yaml

train: ../train/images
val: ../valid/images
test: ../test/images

nc: 2
names: ['skill_success', 'skill_using']

roboflow:
  workspace: pythonproject-3xtlx
  project: skill_s
  version: 5
  license: CC BY 4.0
  url: https://universe.roboflow.com/pythonproject-3xtlx/skill_s/dataset/5

In [ ]:
#Image 싹 긁어 모으기기

%cd /
from glob import glob

img_list = glob('/content/dataset/test/images/*.jpg')
img_list += glob('/content/dataset/train/images/*.jpg')
img_list += glob('/content/dataset/valid/images/*.jpg')
print(len(img_list))

/
752


In [ ]:
#hyperParam

batch_size = 32
num_epoch = 200

In [ ]:
#끌어 모은 이미지 train, test로 split
from sklearn.model_selection import train_test_split
import torch.utils.data as data

train_img_list, test_img_list = train_test_split(img_list, test_size=0.2, random_state=42)
train_img_length, test_img_length = len(train_img_list), len(test_img_list)

train_loader =  data.DataLoader(train_img_list, batch_size=batch_size,
                            shuffle=True, drop_last = True)
test_loader = data.DataLoader(test_img_list, batch_size=batch_size,
                            shuffle=False, drop_last = True)
print(len(train_img_list), len(test_img_list))

601 151


In [ ]:
#train, test에 list up
with open('/content/dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/dataset/test.txt', 'w') as f:
  f.write('\n'.join(test_img_list) + '\n')

In [ ]:
import yaml

with open('/content/dataset/data.yaml', 'r') as f:
  data = yaml.safe_load(f)

print(data)

data['train'] = '/content/dataset/train.txt'
data['val'] = '/content/dataset/test.txt'

with open('/content/dataset/data.yaml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 2, 'names': ['skill_success', 'skill_using'], 'roboflow': {'workspace': 'pythonproject-3xtlx', 'project': 'skill_s', 'version': 5, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/pythonproject-3xtlx/skill_s/dataset/5'}}
{'train': '/content/dataset/train.txt', 'val': '/content/dataset/test.txt', 'test': '../test/images', 'nc': 2, 'names': ['skill_success', 'skill_using'], 'roboflow': {'workspace': 'pythonproject-3xtlx', 'project': 'skill_s', 'version': 5, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/pythonproject-3xtlx/skill_s/dataset/5'}}


In [ ]:
%cd /content/yolov5/
#training img => 총 이미지 사이즈, batch = batch_size, epochs, data => data, 
#!python train.py --batch "{batch_size}" --epochs "{num_epoch}" --data /content/dataset/data.yaml --cfg ./models/yolov5n.yaml --weights yolov5n.pt --name Sivir --optimizer Adam

/content/yolov5


##Detection 

In [ ]:
#이제 디텍션도 안함함
# 코드입력
#!python detect.py --weights ./runs/train/Sivir2/weights/best.pt --conf 0.7 --source /content/drive/MyDrive/Python_project/rofl/test_1.mp4 --save-txt

In [ ]:
#drive mount

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/yolov5/
# 코드입력
#!python detect.py --weights ./runs/train/Sivir2/weights/best.pt --conf 0.7 --source /content/drive/MyDrive/Python_project/rofl/test_2.mp4 --save-txt

In [ ]:
%cd /content/yolov5/
# 코드입력
#!python detect.py --weights ./runs/train/Sivir2/weights/best.pt --conf 0.7 --source /content/drive/MyDrive/Python_project/rofl/test_3.mp4 --save-txt

In [ ]:
%cd /content/yolov5/
# 코드입력
#!python detect.py --weights ./runs/train/Sivir2/weights/best.pt --conf 0.7 --source /content/drive/MyDrive/Python_project/rofl/test_4.mp4 --save-txt

In [ ]:
%cd /content/yolov5/

/content/yolov5


In [ ]:
!python detect.py --weights /content/drive/MyDrive/Colab_Notebooks/PythonProject/Sivir3.pt --conf 0.7 --source /content/drive/MyDrive/Python_project/rofl/test_5.mp4 --save-txt

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
video 1/1 (9307/14303) /content/drive/MyDrive/Python_project/rofl/test_5.mp4: 384x640 (no detections), 208.4ms
video 1/1 (9308/14303) /content/drive/MyDrive/Python_project/rofl/test_5.mp4: 384x640 (no detections), 208.9ms
video 1/1 (9309/14303) /content/drive/MyDrive/Python_project/rofl/test_5.mp4: 384x640 (no detections), 214.7ms
video 1/1 (9310/14303) /content/drive/MyDrive/Python_project/rofl/test_5.mp4: 384x640 (no detections), 207.4ms
video 1/1 (9311/14303) /content/drive/MyDrive/Python_project/rofl/test_5.mp4: 384x640 (no detections), 219.2ms
video 1/1 (9312/14303) /content/drive/MyDrive/Python_project/rofl/test_5.mp4: 384x640 (no detections), 226.5ms
video 1/1 (9313/14303) /content/drive/MyDrive/Python_project/rofl/test_5.mp4: 384x640 (no detections), 209.9ms
video 1/1 (9314/14303) /content/drive/MyDrive/Python_project/rofl/test_5.mp4: 384x640 (no detections), 210.1ms
video 1/1 (9315/14303) /content/drive/MyDrive/Python_project/rofl/test_5.mp4

In [ ]:
%cd /content/yolov5/

!python detect.py --weights /content/drive/MyDrive/Colab_Notebooks/PythonProject/Sivir3.pt --conf 0.7 --source /content/drive/MyDrive/Python_project/rofl/test_6.mp4 --save-txt

In [ ]:
%cd /content/yolov5/

!python detect.py --weights /content/drive/MyDrive/Colab_Notebooks/PythonProject/Sivir3.pt --conf 0.7 --source /content/drive/MyDrive/Python_project/rofl/test_7.mp4 --save-txt
!python detect.py --weights /content/drive/MyDrive/Colab_Notebooks/PythonProject/Sivir3.pt --conf 0.7 --source /content/drive/MyDrive/Python_project/rofl/test_8.mp4 --save-txt
!python detect.py --weights /content/drive/MyDrive/Colab_Notebooks/PythonProject/Sivir3.pt --conf 0.7 --source /content/drive/MyDrive/Python_project/rofl/test_9.mp4 --save-txt
!python detect.py --weights /content/drive/MyDrive/Colab_Notebooks/PythonProject/Sivir3.pt --conf 0.7 --source /content/drive/MyDrive/Python_project/rofl/test_10.mp4 --save-txt
!python detect.py --weights /content/drive/MyDrive/Colab_Notebooks/PythonProject/Sivir3.pt --conf 0.7 --source /content/drive/MyDrive/Python_project/rofl/test_11.mp4 --save-txt

##Analyse

In [16]:
#Detection할 File_setting

import os
import re
import pprint

video_num = 16
detection_set_list = []

weights = 2 #스킬 시전시간이 성공이펙트보다 길기 때문에 주는 가중치

for i in range(video_num):
  label_detected = os.chdir('/content/drive/MyDrive/detect/exp'+str(i+1)+'/labels') #여기서 path 바꿔야할 수도 있음
  file_list = os.listdir()
  detection_set = {} 
  pattern =  r'test_'+str(i+1)+'_(\d+)'
  print('/content/drive/MyDrive/detect/exp'+str(i+1))
  for file_name in file_list:
      file = open(file_name,'r')
      line = file.readline()
      line = line.split(' ')
      detection_class = line[0]

      numbers = re.findall(pattern, file_name)
      if numbers:
        numbers = int(numbers[0])
        detection = {numbers:detection_class}
        detection_set[numbers] = {detection_class}
  detection_set = sorted(detection_set.items())
  detection_set_list.append(detection_set)
  pprint.pprint(detection_set)
for i in range(video_num):
  using = 0
  success = 0
  #counter = 0 #이건 디텍션 확률이 100%가 아니기 때문에 정확도를 수동으로 올려주기위해 사용할 변수임임
  for j in detection_set_list[i]:
    if(j[1]=={'0'}):
      using += 1
    else:
      success += 1
      using += 1
  score = 100*weights*success/using
  print(success)
  print(using)
  print("video nunber: " + str(i+1))
  print("your score: " + str(score)+'%')
#이건 일반적으로 디텍션 확률이 100%일 때 쓸 수 있음

#패턴을 만들어서 추가 대입하면 현실적인 Detection 가능할듯, 시간이 모자라서 여기까지

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 (99, {'0'}),
 (100, {'0'}),
 (101, {'0'}),
 (102, {'0'}),
 (103, {'0'}),
 (104, {'0'}),
 (105, {'0'}),
 (106, {'0'}),
 (107, {'0'}),
 (108, {'0'}),
 (109, {'0'}),
 (110, {'0'}),
 (111, {'0'}),
 (112, {'0'}),
 (113, {'0'}),
 (114, {'0'}),
 (115, {'0'}),
 (116, {'0'}),
 (117, {'0'}),
 (118, {'0'}),
 (119, {'0'}),
 (120, {'0'}),
 (121, {'0'}),
 (122, {'0'}),
 (123, {'0'}),
 (124, {'0'}),
 (125, {'0'}),
 (126, {'0'}),
 (127, {'0'}),
 (128, {'0'}),
 (129, {'0'}),
 (130, {'0'}),
 (131, {'0'}),
 (132, {'0'}),
 (133, {'0'}),
 (134, {'0'}),
 (135, {'0'}),
 (136, {'0'}),
 (137, {'0'}),
 (138, {'0'}),
 (139, {'0'}),
 (140, {'0'}),
 (141, {'0'}),
 (142, {'0'}),
 (143, {'0'}),
 (148, {'0'}),
 (150, {'0'}),
 (152, {'1'}),
 (161, {'0'}),
 (179, {'0'}),
 (180, {'0'}),
 (181, {'0'}),
 (182, {'0'}),
 (183, {'0'}),
 (184, {'0'}),
 (185, {'1'}),
 (186, {'0'}),
 (187, {'1'}),
 (188, {'0'}),
 (189, {'0'}),
 (190, {'0'}),
 (191, {'0'}),
 (193, {'0'}),
 (194, {'0'}),
 (196

In [ ]:
def combine_ones(s):
    # 문자열에서 1만 추출하여 리스트에 저장
    ones = [c for c in s if c == '1']
    # 1이 없는 경우 원본 문자열 반환
    if not ones:
        return s
    # 1을 모두 합쳐 새로운 문자열 생성
    combined = '1' * len(ones)
    return s.replace('1', combined)

def combine_zeros(s):
    # 문자열에서 1만 추출하여 리스트에 저장
    ones = [c for c in s if c == '1']
    # 1이 없는 경우 원본 문자열 반환
    if not ones:
        return s
    # 1을 모두 합쳐 새로운 문자열 생성
    combined = '1' * len(ones)
    return s.replace('1', combined)

def to_list(arr):
  l_return = []
  for i in range(len(arr)):
    l_return.append(arr[i])
  return l_return 


#여기를 사용해서 추가적인 패턴을 넣어볼 예정이였으나 시간이 부족함


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Ingame play

Case


1.   E스킬 사용, 아무것도 못막고 판별도 잘 한 경우

2.   E스킬 사용, 잘 막았고 판별도 잘 한 경우

3.   아무것도 못막았지만 중간에 판별이 이상하게 된 경우

4.   잘 막았지만 중간에 판별이 이상하게 된 경우

5.   처음부터 이상하게 판별한 경우

<일단 Success만 뜸 ==> 시도, 성공 +1>
<일단 Using만 뜸 ==> 시도 +1>

성공/시도

개망했나... 포션, 유성 기타등등 Success만 뜨면 상관 없는데 
Using도 같이 뜸...

0이 안나오면 그 뒤에 1 다 버림
시간이랑 class둘다 필요
